In [ ]:
!pip install matplotlib numpy pandas scikit-learn dask "dask[dataframe]"

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import dask.dataframe as dd
import sklearn as skl
from sklearn.decomposition import PCA

In [ ]:
dataset = '../Datasets/Small_datasetPreprocessed1.parquet'

if os.path.exists(dataset):
    df = pd.read_parquet(dataset)
    df = df.dropna() 
    print(df.head(1))
    print(df.shape)
else:
    print("Dataset not found")


In [ ]:
X = df.drop(['fare_amount'], axis=1)
y = df['fare_amount']

In [ ]:
#LINEAR REGRESSOR MODEL
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

score = model.score(X_test, y_test)
print(score)

In [ ]:
pca = PCA(n_components=2)  # You can specify less if you want to reduce dimensions
# Fit and transform the data
X_pca = pca.fit_transform(X)
X_train_pca, X_test_pca, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

second_linearRegressor = LinearRegression()
second_linearRegressor.fit(X_train_pca, y_train)

y_pred= second_linearRegressor.predict(X_test_pca)

score = second_linearRegressor.score(X_test_pca, y_test)
print("R^2 score: ", score)

x1 = X_test_pca[:, 0]  # First component of X_test_pca
x2 = X_test_pca[:, 1]  # Second component of X_test_pca

# Initialize a 3D plot
fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot(111, projection='3d')

# Plot actual values
ax.scatter(x1, x2, y_test, c='blue', label='Actual y', alpha=0.6, marker='o')

# Plot predictions
ax.scatter(x1, x2, y_pred, c='red', label='Predicted y', alpha=0.6, marker='^')

# Add labels
ax.set_xlabel('x1')
ax.set_ylabel('x2')
ax.set_zlabel('y (Actual and Predicted)')
ax.set_title('3D Plot of Actual vs. Predicted Values')

ax.view_init(elev=0, azim=0)
# Add legend
ax.legend()

# Show plot
plt.show()



In [ ]:
#KNN MODEL
knnModel = KNeighborsRegressor(n_neighbors=3)
knnModel.fit(X_train, y_train)

score = knnModel.score(X_test, y_test)
print(score)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
iterations = 5
scores = []
for i in range(iterations):
  knnModel = KNeighborsRegressor(n_neighbors=(5+2*i))
  knnModel.fit(X_train, y_train)

  score = knnModel.score(X_test, y_test)
  scores.append(score)

print(scores)

In [ ]:
#ELASTICNET MODEL
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train1_scaled = scaler.fit_transform(X_train1)
X_test1_scaled = scaler.transform(X_test1)

scores = []

alpha_values = np.arange(0.5, 1.1, 0.1)  # 0.1 to 1 (inclusive), step of 0.1
l1_ratio_values = np.arange(0.1, 1.1, 0.2)  # 0 to 1 (inclusive), step of 0.1

for alpha in alpha_values:
    for l1_ratio in l1_ratio_values:
        print(f"Alpha: {alpha}, L1 Ratio: {l1_ratio}")
        eNet = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42, max_iter=5000, tol=1e-3)
        eNet.fit(X_train1_scaled, y_train1)
        score = eNet.score(X_test1_scaled, y_test1)
        scores.append(score)
        print(f"Score: {score}")

print(scores)

scores_array = np.array(scores)
print(max(scores_array))

scores_array_reshaped = scores_array.reshape(len(alpha_values), len(l1_ratio_values))
print(scores_array_reshaped)    

In [ ]:
criteria = ['squared_error', 'friedman_mse', 'poisson']

X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.2, random_state=42)

for criterion in criteria:
  treeRegressor = DecisionTreeRegressor(criterion=criterion, splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, random_state=42,)
  treeRegressor.fit(X_train1, y_train1)
  score = treeRegressor.score(X_test1, y_test1)
  print(f"Criterion: {criterion}, Score: {score}")